In [1]:
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0

!pip install folium
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-3.3.0               |           py36_0         747 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [3]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [6]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [9]:
#create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [12]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae"


In [13]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae"


In [14]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,"Central Bay Street, Central Bay Street, Centra..."
1,M2H,North York,"Hillcrest Village, Hillcrest Village, Hillcres..."
2,M4B,East York,"Woodbine Gardens, Parkview Hill, Woodbine Gard..."
3,M1J,Scarborough,"Scarborough Village, Scarborough Village, Scar..."
4,M4G,East York,"Leaside, Leaside, Leaside"
5,M4M,East Toronto,"Studio District, Studio District, Studio District"
6,M1R,Scarborough,"Maryvale, Wexford, Maryvale, Wexford, Maryvale..."
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,"Humber Summit, Humber Summit, Humber Summit"
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


In [15]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

In [16]:
# load the coordinates from the csv file 
coordinates = pd.read_csv("http://cocl.us/Geospatial_data")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae",43.773136,-79.239476


In [19]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,"Central Bay Street, Central Bay Street, Centra...",43.657952,-79.387383
1,M2H,North York,"Hillcrest Village, Hillcrest Village, Hillcres...",43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill, Woodbine Gard...",43.706397,-79.309937
3,M1J,Scarborough,"Scarborough Village, Scarborough Village, Scar...",43.744734,-79.239476
4,M4G,East York,"Leaside, Leaside, Leaside",43.709060,-79.363452
5,M4M,East Toronto,"Studio District, Studio District, Studio District",43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford, Maryvale, Wexford, Maryvale...",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,"Humber Summit, Humber Summit, Humber Summit",43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


## Part 3

In [20]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [31]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [26]:
# filter borough names that contain the word "Toronto"
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    y = x.lower()
    if "toronto" in y:
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [28]:
# create a new DataFrame with only boroughs that contain the word "Toronto"
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar, The Beaches We...",43.668999,-79.315572
3,M4M,East Toronto,"Studio District, Studio District, Studio District",43.659526,-79.340923
4,M4N,Central Toronto,"Lawrence Park, Lawrence Park, Lawrence Park",43.728020,-79.388790


In [46]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [33]:
# define Foursquare Credentials and Version
CLIENT_ID = 'YQWSFRHHQZEUEZE5QL5B1TJNISXS2AD5V5BZ2HJ4SPCMXFMQ' # your Foursquare ID
CLIENT_SECRET = 'NYLBUF2QDSCNOVMLTUQHUYYUQ2UUCJVVIKRGQPGBZ3LFUZPS' # your Foursquare Secret
VERSION = '20191201' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YQWSFRHHQZEUEZE5QL5B1TJNISXS2AD5V5BZ2HJ4SPCMXFMQ
CLIENT_SECRET:NYLBUF2QDSCNOVMLTUQHUYYUQ2UUCJVVIKRGQPGBZ3LFUZPS


### We want to get the top 100 venues that are within a radius of 500 meters.

In [40]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [42]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1688, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [43]:
#check how many venues were returned for each PostalCode
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,"The Beaches, The Beaches, The Beaches",6,6,6,6,6,6
M4K,East Toronto,"The Danforth West, Riverdale, The Danforth West, Riverdale, The Danforth West, Riverdale",43,43,43,43,43,43
M4L,East Toronto,"The Beaches West, India Bazaar, The Beaches West, India Bazaar, The Beaches West, India Bazaar",21,21,21,21,21,21
M4M,East Toronto,"Studio District, Studio District, Studio District",38,38,38,38,38,38
M4N,Central Toronto,"Lawrence Park, Lawrence Park, Lawrence Park",3,3,3,3,3,3
M4P,Central Toronto,"Davisville North, Davisville North, Davisville North",9,9,9,9,9,9
M4R,Central Toronto,"North Toronto West, North Toronto West, North Toronto West",22,22,22,22,22,22
M4S,Central Toronto,"Davisville, Davisville, Davisville",35,35,35,35,35,35
M4T,Central Toronto,"Moore Park, Summerhill East, Moore Park, Summerhill East, Moore Park, Summerhill East",4,4,4,4,4,4


In [44]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 233 uniques categories.


### Analyze venues from each area

In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1688, 236)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",0,0,0,0,0,0,0,0,0,

In [48]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(38, 236)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000

### Create the new dataframe and display the top 10 venues for each PostalCode.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(38, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",Neighborhood,Pub,Asian Restaurant,Health Food Store,Other Great Outdoors,Trail,Diner,Discount Store,Dog Run,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Restaurant,Bookstore,Furniture / Home Store,Indian Restaurant,Fruit & Vegetable Store,Juice Bar
2,M4L,East Toronto,"The Beaches West, India Bazaar, The Beaches We...",Park,Sandwich Place,Pet Store,Pub,Liquor Store,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant
3,M4M,East Toronto,"Studio District, Studio District, Studio District",Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Stationery Store,Fish Market,Coworking Space,Latin American Restaurant,Seafood Restaurant
4,M4N,Central Toronto,"Lawrence Park, Lawrence Park, Lawrence Park",Bus Line,Park,Swim School,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,M4P,Central Toronto,"Davisville North, Davisville North, Davisville...",Pizza Place,Gym,Sandwich Place,Park,Convenience Store,Clothing Store,Breakfast Spot,Food & Drink Shop,Hotel,Doner Restaurant
6,M4R,Central Toronto,"North Toronto West, North Toronto West, North ...",Clothing Store,Coffee Shop,Sporting Goods Shop,Bagel Shop,Gym / Fitness Center,Gift Shop,Mexican Restaurant,Diner,Dessert Shop,Park
7,M4S,Central Toronto,"Davisville, Davisville, Davisville",Sandwich Place,Coffee Shop,Dessert Shop,Pizza Place,Sushi Restaurant,Gym,Café,Italian Restaurant,Fried Chicken Joint,Brewery
8,M4T,Central Toronto,"Moore Park, Summerhill East, Moore Park, Summe...",Gym,Playground,Tennis Court,Trail,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Pub,Light Rail Station,Sports Bar,Supermarket,Sushi Restaurant,Restaurant,Liquor Store,Fried Chicken Joint,Bagel Shop


### Cluster Areas

In [50]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 0, 2, 2, 2, 4, 2], dtype=int32)

In [51]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head()

(38, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031,2,Neighborhood,Pub,Asian Restaurant,Health Food Store,Other Great Outdoors,Trail,Diner,Discount Store,Dog Run,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Restaurant,Bookstore,Furniture / Home Store,Indian Restaurant,Fruit & Vegetable Store,Juice Bar
2,M4L,East Toronto,"The Beaches West, India Bazaar, The Beaches We...",43.668999,-79.315572,2,Park,Sandwich Place,Pet Store,Pub,Liquor Store,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant
3,M4M,East Toronto,"Studio District, Studio District, Studio District",43.659526,-79.340923,2,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Stationery Store,Fish Market,Coworking Space,Latin American Restaurant,Seafood Restaurant
4,M4N,Central Toronto,"Lawrence Park, Lawrence Park, Lawrence Park",43.728020,-79.388790,0,Bus Line,Park,Swim School,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [52]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(38, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,Central Toronto,"Lawrence Park, Lawrence Park, Lawrence Park",43.728020,-79.388790,0,Bus Line,Park,Swim School,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
23,M5P,Central Toronto,"Forest Hill North, Forest Hill West, Forest Hi...",43.696948,-79.411307,1,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
10,M4W,Downtown Toronto,"Rosedale, Rosedale, Rosedale",43.679563,-79.377529,1,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
0,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031,2,Neighborhood,Pub,Asian Restaurant,Health Food Store,Other Great Outdoors,Trail,Diner,Discount Store,Dog Run,Doner Restaurant
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel, Commerce Court...",43.648198,-79.379817,2,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Deli / Bodega,Bakery,Steakhouse,Gym
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville, The Annex...",43.672710,-79.405678,2,Café,Sandwich Place,Coffee Shop,Pharmacy,BBQ Joint,Cosmetics Shop,Pizza Place,Pub,Burger Joint,Liquor Store
25,M5S,Downtown Toronto,"Harbord, University of Toronto, Harbord, Unive...",43.662696,-79.400049,2,Café,Sandwich Place,Bookstore,Restaurant,Japanese Restaurant,Italian Restaurant,Bar,Bakery,French Restaurant,Noodle House
26,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market, Chi...",43.653206,-79.400049,2,Café,Bar,Chinese Restaurant,Vietnamese Restaurant,Mexican Restaurant,Dumpling Restaurant,Coffee Shop,Bakery,Park,Comfort Food Restaurant
27,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,2,Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate
28,M5W,Downtown Toronto,"Stn A PO Boxes 25 The Esplanade, Stn A PO Boxe...",43.646435,-79.374846,2,Coffee Shop,Restaurant,Café,Seafood Restaurant,Hotel,Beer Bar,Bakery,Italian Restaurant,Japanese Restaurant,Art Gallery


### Visualize the clusters 

In [61]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters 

#### Cluster 1 

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Bus Line,Park,Swim School,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 2 

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
10,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 3 

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,2,Neighborhood,Pub,Asian Restaurant,Health Food Store,Other Great Outdoors,Trail,Diner,Discount Store,Dog Run,Doner Restaurant
21,Downtown Toronto,2,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Deli / Bodega,Bakery,Steakhouse,Gym
24,Central Toronto,2,Café,Sandwich Place,Coffee Shop,Pharmacy,BBQ Joint,Cosmetics Shop,Pizza Place,Pub,Burger Joint,Liquor Store
25,Downtown Toronto,2,Café,Sandwich Place,Bookstore,Restaurant,Japanese Restaurant,Italian Restaurant,Bar,Bakery,French Restaurant,Noodle House
26,Downtown Toronto,2,Café,Bar,Chinese Restaurant,Vietnamese Restaurant,Mexican Restaurant,Dumpling Restaurant,Coffee Shop,Bakery,Park,Comfort Food Restaurant
27,Downtown Toronto,2,Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate
28,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Seafood Restaurant,Hotel,Beer Bar,Bakery,Italian Restaurant,Japanese Restaurant,Art Gallery
29,Downtown Toronto,2,Coffee Shop,Café,Hotel,Steakhouse,Restaurant,Seafood Restaurant,American Restaurant,Asian Restaurant,Deli / Bodega,Burger Joint
30,Downtown Toronto,2,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Baby Store,Candy Store,Athletics & Sports,Restaurant
31,West Toronto,2,Bakery,Supermarket,Pharmacy,Café,Athletics & Sports,Bank,Bar,Middle Eastern Restaurant,Music Venue,Gym / Fitness Center


#### Cluster 4

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Garden,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Cluster 5

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4,Gym,Playground,Tennis Court,Trail,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant


### Observation:

I found most of neighborhoods fall into Cluster 3, which mostly contains coffee shop, bakery, hotel and restaurants. In addition, venues in cluster 3 centered at downtown Toronto. Cluster 1 is just busline while Cluster 2 contains parks, trails and playground. Cluster 4 contains garden, dessert shop and restaurant. And cluster 5 are gym and playground.